In [1]:
### Source: https://github.com/bmabey/pyLDAvis/blob/master/notebooks/Gensim%20Newsgroup.ipynb

In [2]:
import pandas as pd
import numpy as np

from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd


In [15]:
clean_mini = pd.read_csv('clean_mini_questions_052320.csv')

In [26]:
clean_mini.head()

,question,summary,tags,user_id,no_of_votes,no_of_answers,date,clean_text,listed_text
0,multiple json objects into a list,"{""Id"": 1,""product"": ""Mobile"",""price"": 32889,""d...","['python', 'json']",diveinsky,0.0,1.0,2019-08-02,multiple json object list,[multiple json object list]
1,How to implement like button concept in viewpa...,I have a viewpager which displays images and b...,"['android', 'android-viewpager']",Marat Zangiev,1.0,1.0,2019-08-02,implement like button concept viewpager,[implement like button concept viewpager]
2,Is it a good practice using Observable with as...,I am using angular 2 common http that return a...,"['javascript', 'angular', 'typescript', 'promi...",Ashwin J Chhetri,48.0,2.0,2019-08-02,good practice use observable async await,[good practice use observable async await]
3,UBSan: Store to misaligned address; what is th...,"I've been running some code under UBSan, and f...",['ubsan'],Flamefire,0.0,1.0,2019-08-02,ubsan store misalign address problem care,[ubsan store misalign address problem care]
4,Django-Filter Form Displaying All Filters?,I've just started working with Django Filter. ...,"['django-filter', 'django-filters']",udeep shrestha,0.0,2.0,2019-08-02,django filter form display filter,[django filter form display filter]


In [18]:
clean_mini.shape

(3769278, 8)

In [8]:
#clean_mini['clean_text'] = clean_mini['clean_text'].tolist()

In [28]:
clean_mini['listed_text'] = [[x] for x in clean_mini['clean_text']]

In [22]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=5, no_above=0.5):
  print('Building dictionary...')
  dictionary = Dictionary(docs)
  stopwords = nltk_stopwords().union(additional_stopwords)
  stopword_ids = map(dictionary.token2id.get, stopwords)
  dictionary.filter_tokens(stopword_ids)
  dictionary.compactify()
  dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
  dictionary.compactify()

  print('Building corpus...')
  corpus = [dictionary.doc2bow(doc) for doc in docs]

  return dictionary, corpus

In [31]:
dictionary, corpus = prep_corpus(clean_mini['listed_text'])

Building dictionary...


TypeError: decoding to str: need a bytes-like object, float found

In [ ]:
MmCorpus.serialize('models/mini_corpus.mm', corpus)
dictionary.save('models/mini_dict.dict')

In [ ]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=50, passes=10)
                                      
lda.save('models/mini_lda.model')

In [ ]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [ ]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)